<a href="https://colab.research.google.com/github/sowmya-cyber/INTERNSHIP-PROJECT-/blob/main/MAININTERNSHIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
#mounting with colab
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import json

In [6]:

# Reading the dataset
dataset=pd.read_csv("/content/gdrive/MyDrive/iris.csv")

In [7]:

#Json file reading and parsing
file_path="/content/gdrive/MyDrive/algaparams.json"
with open(file_path, 'r') as file:
    data = json.load(file)
    print (data)

{'session_name': 'test', 'session_description': 'test', 'design_state_data': {'session_info': {'project_id': '1', 'experiment_id': 'kkkk-11', 'dataset': 'iris_modified.csv', 'session_name': 'test', 'session_description': 'test'}, 'target': {'prediction_type': 'Regression', 'target': 'petal_width', 'type': 'regression', 'partitioning': True}, 'train': {'policy': 'Split the dataset', 'time_variable': 'sepal_length', 'sampling_method': 'No sampling(whole data)', 'split': 'Randomly', 'k_fold': False, 'train_ratio': 0, 'random_seed': 0}, 'metrics': {'optomize_model_hyperparameters_for': 'AUC', 'optimize_threshold_for': 'F1 Score', 'compute_lift_at': 0, 'cost_matrix_gain_for_true_prediction_true_result': 1, 'cost_matrix_gain_for_true_prediction_false_result': 0, 'cost_matrix_gain_for_false_prediction_true_result': 0, 'cost_matrix_gain_for_false_prediction_false_result': 0}, 'feature_handling': {'sepal_length': {'feature_name': 'sepal_length', 'is_selected': True, 'feature_variable_type': 'nu

In [8]:
#preprocessing (Label encoding)
le = LabelEncoder()
dataset['species'] = le.fit_transform(dataset['species'])
dataset.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [9]:
#preprocessing using Standard Scaler
dataset = pd.DataFrame(dataset)
scaler1 = preprocessing.StandardScaler().fit(dataset)
scaled_data=scaler1.transform(dataset)
df=pd.DataFrame(scaled_data)
df.columns=dataset.columns
df.head()
print(df.std(axis=0))

sepal_length    1.00335
sepal_width     1.00335
petal_length    1.00335
petal_width     1.00335
species         1.00335
dtype: float64


In [10]:
# Read target and Regression type
def regression(data):
    target = data['design_state_data']['target']['target']
    rtype = data['design_state_data']['target']['type']
    return target,rtype

In [11]:
#impute Petalwidth
def petal_widthimpu(dataset):
   feature= data['design_state_data']['feature_handling']['petal_width']['feature_details']['missing_values']
   print (feature)
   if feature=="Impute":
     if (dataset['petal_width'].isna().any()) == False:
        print("no missing values and imputation is not needed")
     else:
        dataset['petal_width']=dataset['petal_width']-2
   return dataset

In [12]:
#impute sepal length
def sepal_lengthimpu(dataset):
   feature= data['design_state_data']['feature_handling']['sepal_length']['feature_details']['missing_values']
   print (feature)
   if feature=="Impute":
     if (dataset['sepal_length'].isna().any()) == False:
        print("no missing values and imputation is not needed")
     else:
        dataset['sepal_length']=dataset['sepal_length'].fillna(dataset['sepal_length'].mean())
   return (dataset)

In [13]:
#impute Sepal width
def sepal_widthimpu(dataset):
   feature= data['design_state_data']['feature_handling']['sepal_width']['feature_details']['missing_values']
   print (feature)
   if feature=="Impute":
     if (dataset['sepal_width'].isna().any()) == False:
        print("no missing values and imputation is not needed")
     else:
        dataset['sepal_width']=dataset['sepal_width']-2
   return dataset

In [14]:
#impute petallength
def petal_lengthimpu(dataset):
   feature= data['design_state_data']['feature_handling']['petal_length']['feature_details']['missing_values']
   print (feature)
   if feature=="Impute":
     if (dataset['petal_length'].isna().any()) == False:
        print("no missing values and imputation is not needed")
     else:
        dataset['petal_length']=dataset['petal_length'].fillna(dataset['petal_length'].mean())
   return dataset

In [15]:
 #feature reduction
 def featurereduction(df,frmethod,nooffeatures):
         target,rtype=regression(data)
         print(target)
         print(rtype)
         if frmethod == 'No Reduction':
           return df
         elif frmethod== 'Corr with Target':
           features = df.columns.drop(target)
           X = df[features]
           Y = df[target]
           selector = SelectKBest(f_regression, k=nooffeatures)
           X_new = selector.fit_transform(X, Y)
           selectedfeatures = X.columns[selector.get_support()]
           return df[selectedfeatures]
         elif frmethod=="Tree-based":
           features = df.columns.drop(target)
           X = df[features]
           Y = df[target]
           model = DecisionTreeRegressor()
           model.fit(X, Y)
           feature_importances = pd.Series(model.feature_importances_, index=X.columns)
           selectedfeatures = feature_importances.nlargest(nooffeatures).index
           print (selectedfeatures)
           return df[selectedfeatures]

In [18]:
def Evaluation(prediction1,prediction2): # evaluate the performance metrics base on MSE and RMSE and graph is plotted

  mse = mean_squared_error(Ytest, prediction1)
  rmse = mse**.5
  print(mse)
  print(rmse)
  mse = mean_squared_error(Ytest, prediction2)
  rmse = mse**.5
  print(mse)
  print(rmse)
  x_ax = range(len(Ytest))
  plt.plot(x_ax, Ytest, lw=0.6, color="blue", label="original")
  plt.plot(x_ax,prediction2,lw=0.8, color="red", label="predicted")
  plt.title("test and predicted data")
  plt.legend()
  plt.show()

In [19]:
def elasticnet():#Elasticnet Regression
  print("The selected regression is Elasticnet Regression ")
  params = {
    "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "l1_ratio": [0, 0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.99, 1],
}
  elasticnet = ElasticNet()
  elasticnet.fit(Xtrain,Ytrain)
  folds=3
  prediction1 = elasticnet.predict(Xtest)
  grid_cv_model = GridSearchCV(estimator=elasticnet,
                       param_grid=params,
                       scoring='neg_mean_absolute_error',
                       cv=folds,
                       return_train_score=True,
                       verbose=1)
  grid_cv_model.fit(Xtrain,Ytrain)

  prediction2 = grid_cv_model.predict(Xtest)
  Evaluation(prediction1,prediction2)


In [20]:
def RandomFos(): #Randomforest Regression
  print("The selected regression is Randomforest Regression ")
  rf=RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth = 5, random_state = 18)
  rf.fit(Xtrain,Ytrain)
  grid = {
    'n_estimators': [100,200,300,400],
    'max_features': ['sqrt','log2'],
    'max_depth' : [2,3,4,5,6],
    'random_state' : [12,18]
     }
  CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5)
  CV_rfr.fit(Xtrain, Ytrain)
  prediction1 = rf.predict(Xtest)
  mse = mean_squared_error(Ytest, prediction1)
  rmse = mse**.5
  print(mse)
  print(rmse)
  prediction2 = CV_rfr.predict(Xtest)
  print("The selected regression is Randomforest Regression ")
  Evaluation(prediction1,prediction2)


In [34]:
def linearregression(): #linear Regression
    print("The selected regression is Linear Regression ")
    lr=LinearRegression()
    lr.fit(Xtrain,Ytrain)
    param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False]
}
    grid = GridSearchCV(estimator=lr, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
    grid.fit(Xtrain,Ytrain)
    prediction1=lr.predict(Xtest)
    prediction2=grid.predict(Xtest)
    Evaluation(prediction1,prediction2)




In [22]:
def GBR(): #Gradientboost Regression
 print("The selected regression is Gradientboost Regression ")
 rg=GradientBoostingRegressor()
 rg.fit(Xtrain, Ytrain)
 param_grid = {'learning_rate': [0.2,0.02,0.02,1],
              'max_depth'    : [2,4,6,8,10]
                 }
 gridmodel = GridSearchCV(estimator=rg, param_grid=param_grid, cv=3, n_jobs = -1)
 gridmodel.fit(Xtrain, Ytrain)
 prediction1=rg.predict(Xtest)
 prediction2=gridmodel.predict(Xtest)
 Evaluation(prediction1,prediction2)


In [18]:
def Lassoregu(): #Lasso Regression
 print("Lasso Regression")
 params = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1.0, 2.0, 5.0, 10., 20.],
    'selection': ['random', 'cyclic']
 }
 model = Lasso()
 model.fit(Xtrain, Ytrain)
 folds=3
 grid_cv_model = GridSearchCV(estimator=model,
                       param_grid=params,
                       scoring='neg_mean_absolute_error',
                       cv=folds,
                       return_train_score=True,
                       verbose=1)
 grid_cv_model.fit(Xtrain, Ytrain)
 prediction1 = model.predict(Xtest)
 prediction2 = grid_cv_model.predict(Xtest)
 print("The selected regression is Lasso Regression ")
 Evaluation(prediction1,prediction2)

In [23]:
def xgboost(): #XGBoost Regression
  print("XGBoost Regression")
  model = xgb.XGBRegressor()
  model.fit(Xtrain, Ytrain)
  parameters = {'nthread':[4],'learning_rate': [.03, 0.05, .07],'max_depth': [5, 6, 7],'min_child_weight': [4],'silent': [1],'subsample': [0.7],'colsample_bytree': [0.7],
                  'n_estimators': [500]}
  gridmodel = GridSearchCV( model,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)
  gridmodel.fit(Xtrain, Ytrain)
  prediction1 = model.predict(Xtest)
  prediction2 = gridmodel.predict(Xtest)
  print("The selected regression is XG Boost Regression ")
  Evaluation(prediction1,prediction2)

In [24]:
def DecisionTreeRegressor():#Decisiontree Regression
 print("Decisiontree Regression")
 model = DecisionTreeRegressor(random_state = 0)
 model.fit(Xtrain, Ytrain)
 prediction1=model.predict(Xtest)
 param_grid = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
 }
 grid = GridSearchCV(DecisionTreeRegressor(),param_grid, cv=5)
 grid.fit(Xtrain,Ytrain)
 prediction2=grid.predict(Xtest)
 print("The selected regression is Decisiontree Regression ")
 Evaluation(prediction1,prediction2)


In [25]:
def etr(): #ExtraTreesRegression
  print("ExtraTreesRegressor")
  etr = ExtraTreesRegressor()
  etr.fit(Xtrain, Ytrain)
  param_grid={
        'n_estimators': range(50,126,25),
        'max_features': range(50,401,50),
        #'m in_samples_leaf': range(20,50,5),
        #'min_samples_split': range(15,36,5),
   }
  cv = 3
  param_grid = {'n_estimators': [10, 50, 100, 200]}
  gridmodel = GridSearchCV(estimator=etr, param_grid=param_grid, cv=cv)
  gridmodel.fit(Xtrain, Ytrain)
  prediction2 = gridmodel.predict(Xtest)
  prediction1=etr.predict(Xtest)
  print("The selected regression is Extratree Regression ")
  Evaluation(prediction1,prediction2)


In [26]:
def SVMRE():#SVR Regression
   print("SVR Regression")
   regressor = SVR(kernel='rbf')
   regressor.fit(Xtrain,Ytrain)
   prediction1=regressor.predict(Xtest)
   parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.5, 10],'gamma': [1e-7, 1e-4],'epsilon':[0.1,0.2,0.5,0.3]}
   gridmodel = GridSearchCV(estimator=regressor, param_grid=parameters)
   gridmodel.fit(Xtrain, Ytrain)
   prediction2=gridmodel.predict(Xtest)
   print("The selected regression is SVM Regression ")
   Evaluation(prediction1,prediction2)

In [27]:
if __name__ == "__main__":
# Read target and prediction type
  target,rtype=regression(data)
  print(target)
  print(rtype)

petal_width
regression


In [16]:
#apply imputation
sepal_lengthimpu(dataset)
sepal_widthimpu(dataset)
petal_widthimpu(dataset)
petal_lengthimpu(dataset)

Impute
no missing values and imputation is not needed
Impute
no missing values and imputation is not needed
Impute
no missing values and imputation is not needed
Impute
no missing values and imputation is not needed


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [17]:
#apply feature selection
featurered = data['design_state_data']['feature_reduction']
featurereduction(df,featurered['feature_reduction_method'],int(featurered['num_of_features_to_keep']))

petal_width
regression
Index(['petal_length', 'species', 'sepal_width', 'sepal_length'], dtype='object')


,petal_length,species,sepal_width,sepal_length
0,-1.341272,-1.224745,1.032057,-0.900681
1,-1.341272,-1.224745,-0.124958,-1.143017
2,-1.398138,-1.224745,0.337848,-1.385353
3,-1.284407,-1.224745,0.106445,-1.506521
4,-1.341272,-1.224745,1.263460,-1.021849
...,...,...,...,...
145,0.819624,1.224745,-0.124958,1.038005
146,0.705893,1.224745,-1.281972,0.553333
147,0.819624,1.224745,-0.124958,0.795669
148,0.933356,1.224745,0.800654,0.432165


In [28]:
#spliting training and testing
from sklearn.model_selection import train_test_split
features = df.columns.drop(target)
X = df[features]
Y = df[target]
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.25,random_state=2)

In [ ]:
#run and predict each modelbased on the metric mean square error and root mean square error.Here the run and predicted values are evaluated and plotted a graph based on that
rtype = data['design_state_data']['target']['type']
if rtype=="regression":
  regress=data['design_state_data']['algorithms']
  for key in regress:
   if key=="RandomForestRegressor":RandomFos()
   if key=="LinearRegression":linearregression()
   if key=="GBTRegressor":GBR()
   if key=="LassoRegression":Lassoregu()
   if key=="ElasticNetRegression":elasticnet()
   if key=="xg_boost":xgboost()
   if key=="DecisionTreeRegressor":DecisionTreeRegressor()
   if key=="extra_random_trees":etr()
   if key=="SVM":SVMRE()